In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from munch import Munch

In [ ]:
config = Munch()

In [ ]:
phases = ["train", "validation"]
datasets = Munch({phase: DRLDataset(**config.data[phase]) for phase in phases})
data_loader = Munch(
    {
        phase: DataLoader(
            datasets[phase],
            batch_size=config.train.batch_size,
            num_workers=4,
        )
        for phase in phases
    }
)

model = HadarmardFusionNet(datasets.train, **config.model)
model.to(device)
criterion = nn.BCELoss(reduction="sum")
optimizer = torch.optim.Adam(model.parameters(), lr=config.train.lr)


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
class Train(nn.Module):
    def __init__(self):
        super().__init__(self)

    def training_step(self, batch, batch_idx):
        if self.freeze == "all":
            model.image_encoder.eval()
        images, questions, answers = batch
        out = self(images, questions)
        loss = criterion(out, answers)
        self.log("train_loss", loss)
        self.log("train_accuracy", accuracy_score(answers, out))
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
def train(config):
    result = Munch()
    for epoch in trange(config.train.num_epochs):
        for phase in phases:
            result[phase] = Munch()
            result[phase].total_loss = 0
            result[phase].num_correct = 0
            for batch in data_loader[phase]:
                step(
                    model,
                    criterion,
                    optimizer,
                    phase,
                    batch,
                    result,
                    config.train.freeze,
                    device,
                )
        report_result(epoch, phases, result, data_loader)
        torch.save(model.state_dict(), "model.pt")

In [ ]:
train(config)